# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

This is a classification problem. as we are making a model that will either output yes or no (a discrete variable).

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [163]:
# Import libraries
import numpy as np
import pandas as pd

In [164]:
# Read student data
student_data = pd.read_csv("student-data.csv")

print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [166]:
from __future__ import division

n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1 #subtract the label
n_passed = student_data[student_data["passed"] == "yes"].shape[0]
n_failed = student_data[student_data["passed"] == "no"].shape[0]
grad_rate = 100 * (n_passed / n_students) 

print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [167]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label

print("Feature columns")
print(feature_cols)
print

print("Target column")
print(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels

# display the first 5 rows
X_all.head()  

Feature columns
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column
passed


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4


### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [168]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            # e.g. 'school' => 'school_GP', 'school_MS'
            col_data = pd.get_dummies(col_data, prefix=col)  

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [180]:
from sklearn.cross_validation import train_test_split

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.24, random_state=12)

# Note: If you need a validation set, extract it from within training data
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [170]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here

#clf?
#print clf

Training RandomForestClassifier...
Done!
Training time (secs): 0.023


In [171]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.992481203008


In [172]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.704


In [189]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

def evaluate_model(clf, X_train, y_train, X_test, y_test):
    """ Evaluate a model by testing it on varying training set sizes
    """
    for n in [100, 200, 300]:
        # get the n first rows from the dataframe
        X_subset = X_train.head(n)
        y_subset = y_train.head(n) 
        # train and score the model
        train_predict(clf, X_subset, y_subset, X_test, y_test)       
    
evaluate_model(clf, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.884057971014
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.761904761905
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.883720930233
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.8
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.008
Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.87610619469
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.8


### Support Vector Machine

In [195]:
from sklearn import svm

# Initialize SVM classifier with default parameter settings
svm_classifier = svm.SVC()

evaluate_model(svm_classifier, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.884057971014
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.761904761905
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.883720930233
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.8
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.87610619469
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.8


### Multinomial Naive Bayes

In [183]:
from sklearn.naive_bayes import MultinomialNB

# Initialize MultinomialMB with default parameter settings
nb = MultinomialNB()

evaluate_model(nb, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training MultinomialNB...
Done!
Training time (secs): 0.012
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.828125
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.666666666667
------------------------------------------
Training set size: 200
Training MultinomialNB...
Done!
Training time (secs): 0.002
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.810996563574
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.751879699248
------------------------------------------
Training set size: 300
Training MultinomialNB...
Done!
Training time (secs): 0.001
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.793503480278
Predicting labels using MultinomialNB...


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

### Fine-tuning the model

In [193]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

clf = svm.SVC()

param_grid = [
  {'C': [1, 10, 100], 'kernel': ['linear']},
  {'C': [1, 10, 100], 'degree': [2, 3, 4], 'kernel': ['poly']},
  {'C': [1, 10, 100], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

# use f1_score as the scoring function
scoring_function = make_scorer(f1_score, pos_label='yes')

# set up the grid search cross-validation
gs = GridSearchCV(clf, param_grid, scoring_function, cv=10)

# run grid search
gs.fit(X_train, y_train)

# keep the best model
model = gs.best_estimator_

# report the best score
print("Best Score")
print(gs.best_score_)
print("")

# report the grid scores
print("Grid Scores")
sorted(gs.grid_scores_, key=lambda x: x.mean_validation_score, reverse=True)

Best Score
0.806722076435

Grid Scores


[mean: 0.80672, std: 0.02426, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.0001},
 mean: 0.80333, std: 0.02650, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001},
 mean: 0.80000, std: 0.00000, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001},
 mean: 0.79755, std: 0.00735, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001},
 mean: 0.79510, std: 0.00980, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.001},
 mean: 0.78840, std: 0.05500, params: {'kernel': 'poly', 'C': 1, 'degree': 2},
 mean: 0.78564, std: 0.04288, params: {'kernel': 'linear', 'C': 100},
 mean: 0.78233, std: 0.04558, params: {'kernel': 'linear', 'C': 10},
 mean: 0.77720, std: 0.05733, params: {'kernel': 'rbf', 'C': 100, 'gamma': 0.001},
 mean: 0.77153, std: 0.03593, params: {'kernel': 'linear', 'C': 1},
 mean: 0.74429, std: 0.06237, params: {'kernel': 'poly', 'C': 10, 'degree': 2},
 mean: 0.72942, std: 0.08005, params: {'kernel': 'poly', 'C': 1, 'degree': 3},
 mean: 0.71742, std: 0.08440, params: {'kernel': 'poly', 'C'

In [194]:
# Evaluate our final model
evaluate_model(model, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.848920863309
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.73381294964
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.831746031746
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.797297297297
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.828025477707
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.810810810811
